<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Data_Augmentation_and_Generation(Prasoon%20Approach).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Parasite.7z
!7z x /content/Parasite.7z

--2021-05-08 05:05:05--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Parasite.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20003284 (19M) [application/octet-stream]
Saving to: ‘Parasite.7z’

Parasite.7z         100%[===================>]  19.08M  72.1MB/s    in 0.3s    

2021-05-08 05:05:07 (72.1 MB/s) - ‘Parasite.7z’ saved [20003284/20003284]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 20003284 bytes (20 MiB)

Extracting archive: /content/Parasite.7z
--
Path = /content/Parasite.7z
Typ

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

import torchvision
from PIL import Image

In [ ]:
del data

In [5]:
data = pd.read_csv("/content/Dataset/all_meta_data_updated.csv")

In [6]:
data.species.replace({"Hymenolepis diminuta":"Hymenolepis spp",
                    "Hymenolepis nana":"Hymenolepis spp",
                    # "Taenia solium":"na",
                    # "Clonorchis sinensis":"na",
                    # "Trichinella sp.":"na",
                    # "B. coli":"na",
                    # "Toxoplasma gondii":"na",
                    },inplace = True)
data['phylum'].replace({'Roundworms':'Nematoda','Flatworm':'Platyhelminthes'},inplace = True)
data['class'].replace({'\tConoidasida':'Conoidasida','Tapeworms':'Cestoda'},inplace = True)
data['species'].replace({'Entamoeba histolytica]':'Entamoeba histolytica'},inplace = True)
data['form'].replace({'rings':'trophozoites','decorticated egg':'egg','ookinete':'egg'},inplace = True)
data['sample'].replace({'intestinal tissue':'intestine', 'intestine ':'intestine','sputum':'lungs'},inplace = True)

In [7]:
data['species'].value_counts()

Plasmodium falciparum          36
Plasmodium vivax               36
Cyclospora cayetanensis        30
Plasmodium malariae            30
Ascaris lumbricoides           29
Plasmodium ovale               27
Entamoeba histolytica          27
Giardia duodenalis             27
Strongyloides stercoralis      22
Hymenolepis spp                21
Dibothriocephalus latus        21
Trichuris Trichuria            19
Enterobius vermicularis        18
Toxocara canis                 17
Ancyclostoma sp.               17
Cryptosporidium sp.            16
Sarcocystis sp.                16
Plasmodium knowlesi            15
Trichinella sp.                13
Toxoplasma gondii              13
Echinococcus granulosus        12
Fasciola hepatica              11
B. coli                         9
Clonorchis sinensis             8
Dicrocoelium dendriticum        7
Taenia solium                   6
Echinococcus multilocularis     4
Strongyloides fuelleborni       3
Name: species, dtype: int64

In [8]:
data = [x for _,x in data.groupby(['phylum', 'class', 'genus', 'species', 'form', 'sample']) if len(x)>=8]

data = pd.concat(data,ignore_index=True)

In [9]:
data = data[data['sample']!='na']
data = data[data['sample']!='liver']

In [10]:
labels = ['phylum', 'class', 'genus', 'species', 'form', 'sample']

In [11]:
train, val = train_test_split(
    data, test_size=0.25, random_state=0,stratify = data['species'])

In [12]:
for col in labels:
  print(15*'-'+col+15*'-')
  print(set(train[col].unique())-set(val[col].unique()))
  print(set(val[col].unique())-set(train[col].unique()))
  print(train[col].nunique(),val[col].nunique(),sep = ' -> ')
  print(val[col].nunique(),train[col].nunique(),sep = ' -> ')

---------------phylum---------------
set()
set()
5 -> 5
5 -> 5
---------------class---------------
set()
set()
7 -> 7
7 -> 7
---------------genus---------------
set()
set()
11 -> 11
11 -> 11
---------------species---------------
set()
set()
15 -> 15
15 -> 15
---------------form---------------
set()
set()
7 -> 7
7 -> 7
---------------sample---------------
set()
set()
4 -> 4
4 -> 4


In [ ]:
del test,val2

In [13]:
val2, test = train_test_split(
    val, test_size=0.50, random_state=0,stratify = val['species'])

In [ ]:
val['form'].value_counts()

trophozoites    20
egg             18
oocysts         15
adult           12
gametocyte      10
larva            9
cyst             9
schizont         6
sarcocysts       1
Name: form, dtype: int64

In [14]:
for col in labels:
  print(15*'-'+col+15*'-')
  print(set(val2[col].unique())-set(test[col].unique()))
  print(set(test[col].unique())-set(val2[col].unique()))
  print(val2[col].nunique(),test[col].nunique(),sep = ' -> ')
  print(test[col].nunique(),val2[col].nunique(),sep = ' -> ')

---------------phylum---------------
set()
set()
5 -> 5
5 -> 5
---------------class---------------
set()
set()
7 -> 7
7 -> 7
---------------genus---------------
set()
set()
11 -> 11
11 -> 11
---------------species---------------
set()
set()
15 -> 15
15 -> 15
---------------form---------------
set()
set()
7 -> 7
7 -> 7
---------------sample---------------
set()
set()
4 -> 4
4 -> 4


In [15]:
for path in train['species'].unique():
  os.makedirs(f"/content/train/{path}")


for path in val['species'].unique():
  os.makedirs(f"/content/val/{path}")

for path in test['species'].unique():
  os.makedirs(f"/content/test/{path}")

In [ ]:
!rm -r train val test

In [16]:
t = []
t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomPerspective(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomVerticalFlip(p=1)]))


t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomVerticalFlip(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomPerspective(p=1)]))

In [17]:
def transform_save(data,index,folder):
  img = Image.open(data[8])
  images = [img]
  images.extend([T(img) for T in t])
  image_name = data[6].split('.')[0]
  for i, image in enumerate(images):
    image.convert('RGB').save(f"/content/{folder}/{data[3]}/{image_name}_{i}_{index}.jpg")
  values = [[data[0],data[1],data[2],data[3],data[4],data[5],f"./{folder}/{data[3]}/{image_name}_{i}_{index}.jpg"] for i in range(len(images))]
  return values


def get_dataframe(dataframe,folder = 'train'):
  new = []
  for i in dataframe.index:
    new.extend(transform_save(dataframe.loc[i,:].values,index =i,folder=folder))
  return pd.DataFrame(new,columns=['phylum','class','genus','species','form','sample','img_path'])

In [18]:
train = get_dataframe(train)

In [19]:
val = get_dataframe(val,folder = 'val')

In [20]:
test = get_dataframe(test,folder = 'test')

In [21]:
train.to_csv('/content/train/train.csv',index = False)
val.to_csv('/content/val/val.csv',index = False)
test.to_csv('/content/test/test.csv',index = False)

In [22]:
!7z a TrainP.7z train
!7z a ValP.7z val
!7z a TestP.7z test


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive:
  0M Scan           16 folders, 1141 files, 11427210 bytes (11 MiB)

Creating archive: TrainP.7z

Items to compress: 1157

  0%     12% 222 + train/Cyclospora cayetanensis/Cyclospora_wetmount_HB1_2_159.jpg                                                                           27% 373 + train/Enterobius vermicularis/E_vermicularis_eggs_tissue2_3_230.jpg                                                                               41% 533 + train/Hymenolepis sp

In [23]:
labels = ['phylum', 'class', 'genus', 'species', 'form', 'sample']
train = pd.read_csv("/content/train/train.csv")
# train = train[train['sample']!='na']
# train = train[train['sample']!='liver'] 
val = pd.read_csv("/content/val/val.csv")
test = pd.read_csv("/content/test/test.csv")
# val = val[val['sample']!='na']
# val = val[val['sample']!='liver']
for col in labels:
  print(15*'-'+col+15*'-')
  print(train[col].nunique(),val[col].nunique(),test[col].nunique(),sep=' -> ')

---------------phylum---------------
5 -> 5 -> 5
---------------class---------------
7 -> 7 -> 7
---------------genus---------------
11 -> 11 -> 11
---------------species---------------
15 -> 15 -> 15
---------------form---------------
7 -> 7 -> 7
---------------sample---------------
4 -> 4 -> 4
